In [1]:
#Function to get the wikipedia article
from wikipydia import dict_storage, wikidb, parse, wikisyn
from wiki_sections import get_article_obj
wiki_db = wikidb.WikiDb()

def get_article(href):
    
    article, downloaded = wiki_db.get_article_by_href(href)
    
    if downloaded:
        wiki_db.save()
        print("Db updated.")
    
    return get_article_obj(article.title(), article.html())

In [2]:
#Function to get the keywords from a piece of text

from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

stop_words = set(stopwords.words('english')).union(set(punctuation)).union(set(["''", "``"]))

def get_text_keywords(text, n=10):
    tokens = word_tokenize(text.lower())
    token_counter = Counter()
    
    for word in tokens:
        if len(word) <= 1:
            continue
        if word in stop_words:
            continue
    
        token_counter[word] += 1
    
    #return token_counter.most_common(n)
    
    return [w for w, count in token_counter.most_common(n)]

#get_text_keywords("Linear algebra is the branch of mathematics concerning vector spaces and linear mappings between such spaces. It includes the study of lines, planes, and subspaces, but is also concerned with properties common to all vector spaces.")

In [3]:
#Function to search wikipedia articles

import requests

def search_wikipedia(term, lang="en", timeout=60):
    """Function search articles on wikipedia."""
    #import exceptions

    #https://en.wikipedia.org/w/api.php?action=query&utf8&list=search&srsearch=neural
    #https://www.mediawiki.org/wiki/API:Search

    # https://en.wikipedia.org/w/api.php?action=parse&redirects&page=fluid_mechanics

    req_params = [
        'action=query',
        'utf8',
        'list=search',
        'format=json',
        'srwhat=text',
        'srprop',
        'srlimit=500',
        'srsearch=' + term
    ]

    wikipedia_api_url = "https://" + lang + ".wikipedia.org/w/api.php?" + "&".join(req_params)

    page_data = requests.get(wikipedia_api_url, timeout=timeout).json()

    results = [result['title'] for result in page_data['query']['search']]
    
    return results

#search_wikipedia("neural")

In [4]:
def test_indexes():
    pages_to_download = [i for i in range(67)]
    batch_size = 4
    print(pages_to_download)
    print("\n")
    
    for i in range(0, len(pages_to_download), batch_size):
        print(pages_to_download[i:i+batch_size])

    #for i in range(ceil(len(pages_to_download) / batch_size)):
        #print(pages_to_download[i * batch_size : (i+1) * batch_size])

#test_aee()    

In [5]:
DictStorage = dict_storage.DictStorage
wiki_links = DictStorage("wiki_links")
from collections import defaultdict
from urllib.parse import quote

from math import ceil

def get_wikipedia_links(pages, batch_size=20):
    
    pages_to_download = list()
    
    for page in pages:
        if page not in wiki_links:
            pages_to_download.append(page)
    
    #If there is pages not locally present, download and save data
    if len(pages_to_download) > 0:
        for i in range(0, len(pages_to_download), batch_size):
            #Query all the pages not present
            downloaded_links = query_wikipedia_links(pages_to_download[i:i+batch_size])
            for t, l in downloaded_links.items():
                wiki_links[t] = l
        wiki_links.save()
    
    links_to_return = dict()
    for page in pages:
        try:
            links_to_return[page] = wiki_links[page]
        except KeyError:
            print("Not found links for: " + page)
        
    return links_to_return
    
def query_wikipedia_links(pages, lang="en", timeout=60):
    
    links = defaultdict(set)
    plcontinue = None
    
    while True:
        print("Querying plcontinue: " + str(plcontinue))
        
        partial_links, plcontinue = partial_query_wikipedia_links(pages, plcontinue)
        
        for title, link_set in partial_links.items():
            links[title] = links[title].union(link_set)

        if not plcontinue:
            break;
            
    return links
        

def partial_query_wikipedia_links(pages, plcontinue=None, lang="en", timeout=60):
    #https://en.wikipedia.org/w/api.php?action=query&titles=MQTT&prop=links&pllimit=500&pltitles=Adafruit|Internet%20layer
    
    req_params = [
        'action=query',
        'utf8',
        'prop=links',
        'plnamespace=0',
        'format=json',
        'pllimit=500',
        'titles=' + "|".join([quote(page) for page in pages])
    ]
    
    if plcontinue:
        req_params.append("plcontinue=" + plcontinue)

    wikipedia_api_url = "https://" + lang + ".wikipedia.org/w/api.php?" + "&".join(req_params)

    page_data = requests.get(wikipedia_api_url, timeout=timeout).json()

    
    #If there is a continue page
    if "continue" in page_data:
        plcontinue_param = page_data['continue']['plcontinue']
    else:
        plcontinue_param = None
        
    links = defaultdict(set)
    
    for pageid, data in page_data['query']['pages'].items():
        page_title = data["title"]
        #links[page_title] = set()
        if "links" not in data:
            continue
        for l in data['links']:
            links[page_title].add(l['title'])
            
    return links, plcontinue_param
        
    
def test_query_wikilinks1():
    
    test_query_links = get_wikipedia_links(['Deep learning',
      'Feature learning',
      'Artificial neural network',
      'Convolutional neural network',
      'Recurrent neural network',
      'Machine learning',
      'Autoencoder',
      'Hierarchical temporal memory',
      'Pattern recognition',
      'Sepp Hochreiter',
      'sofuasduasjd',
      "MQTT",
      "Linear algebra"])

    print("")
    for page, links in test_query_links.items():
        print(page, len(links))
        
def test_query_wikilinks2():
    print(wiki_links.items())
    
#test_query_wikilinks1()

In [6]:
def get_sections_text(href):
    sections_text = list()
    art = get_article(href)
    
    for sec in art.flatten_sections():
        if sec.content and not sec.content.isspace(): 
            sections_text.append(sec.content)
    
    #for sec in art:
        #sections_text.append(str(sec).replace("\n", " "))
    return sections_text

#len(get_sections_text("MQTT"))

In [7]:
def get_sections_titles(href):
    sections_titles = list()
    art = get_article(href)
    
    for sec in art.flatten_sections():
        sections_titles.append(sec.title)
    
    return sections_titles
#get_sections_titles("Deep_learning")

In [8]:
def get_sections_articles_candidates(href):
    
    #1. Get article sections text
    sections_text = get_sections_text(href)
    print(sections_text)
    #2. Get sections keywords
    sections_keywords = list()
    for sec_text in sections_text:
        sections_keywords.append(get_text_keywords(sec_text))
        
    #3. Get articles sugestions for each section
    sections_articles_candidates = list()
    for sec_keywords in sections_keywords:

        search_term = " ".join(word for word in sec_keywords)
        sections_articles_candidates.append(search_wikipedia(search_term)[:10])
    
    return sections_articles_candidates   
#get_sections_articles_candidates("MQTT")

In [37]:
def get_sections_articles_candidates_by_titles(href):
    
    #1. Get article sections text
    sections_titles = get_sections_titles(href)
        
    #2. Get articles sugestions for each section
    sections_articles_candidates = list()
    #sections_articles_candidates = dict()
    for title in sections_titles:
        sections_articles_candidates.append({title: search_wikipedia(title)[:10]})
        #sections_articles_candidates[title] = search_wikipedia(title)
    
    return sections_articles_candidates   
#get_sections_articles_candidates_by_titles("MQTT")

In [38]:
from networkx import DiGraph

def get_graph_from_candidates(sec_candidates):
    
    #1. Create graph
    graph = DiGraph()
    
    #2. Add nodes for each set of candidates
    for candidates in sec_candidates:
        for art in candidates:
            graph.add_node(art)
    
    #3. Get links from all the graph nodes
    nodes_links = get_wikipedia_links(graph.nodes())
    
    #4. Add links in the graph if the target already exists in the graph
    for node, links in nodes_links.items():
        for link in links:
            #Avoid loopback links and targets that does not belongs to the graph
            if node!=link and graph.has_node(link):
                graph.add_edge(node, link)
        
    return graph

In [11]:
def get_edges_from_lists(sources, targets, graph):
    edges_list = list()
    for source in sources:
        for target in targets:
            if graph.has_edge(source, target):
                edges_list.append((source, target))
    return edges_list            

In [12]:
def get_edges_rank(sec_candidates, graph):
    edges_rank = Counter()
    prev_set = set(sec_candidates[0])
    
    for candidates in sec_candidates[1:]:
        edges_rank.update(get_edges_from_lists(candidates, prev_set, graph))
        prev_set = prev_set.union(set(candidates))
        
    return edges_rank    

In [41]:
sec_candidates = get_sections_articles_candidates_by_titles("Artificial_neural_networks")

In [42]:
sec_candidates

[{'Artificial neural network': ['Artificial neural network',
   'Fast Artificial Neural Network',
   'Biological neural network',
   'Feedforward neural network',
   'Quantum neural network',
   'Neural network software',
   'Neural network (disambiguation)',
   'Modular neural network',
   'Recurrent neural network',
   'Convolutional neural network']},
 {'History': ['History',
   'HIStory: Past, Present and Future, Book I',
   'World history',
   'History of Turkey',
   'Alternate history',
   'Philosophy of history',
   'Medical history',
   'Social history',
   'History of technology',
   'Economic history']},
 {'Hebbian learning': ['Hebbian theory',
   'Anti-Hebbian learning',
   'Unsupervised learning',
   'Associative sequence learning',
   'GeneRec',
   'Generalized Hebbian Algorithm',
   'Leabra',
   'Competitive learning',
   'Spike-timing-dependent plasticity',
   'Long-term potentiation']},
 {'Backpropagation and resurgence': ['Artificial neural network',
   'Deep learning'

In [18]:
graph = get_graph_from_candidates(sec_candidates)

In [19]:
edges_rank = get_edges_rank(sec_candidates, graph)

In [20]:
edges_rank.most_common()

[(('Software developer', 'Computer hardware'), 2),
 (('Free software', 'Hardware restriction'), 2),
 (('Application software', 'Software development'), 2),
 (('Application software', 'Integrated development environment'), 2),
 (('Software development', 'Computer hardware'), 2),
 (('Application software', 'Free software'), 2),
 (('Free software', 'Open-source hardware'), 2),
 (('Software categories', 'Software development'), 1),
 (('Open-source software', 'Collaborative software development model'), 1),
 (('Arduino', 'Intel Galileo'), 1),
 (('Software industry', 'Software development'), 1),
 (('Open-source software', 'Free software'), 1),
 (('Software engineer', 'Software development process'), 1),
 (('Arduino', 'List of Arduino boards and compatible systems'), 1),
 (('Software engineering', 'Software prototyping'), 1),
 (('Free software', 'Open-source software development'), 1),
 (('Software development', 'Custom software'), 1),
 (('Application virtualization', 'Integrated development 

In [ ]:
keep studying the approach of order of information presentation